In [ ]:
import pyparsing  # make sure you have this installed
import numpy as np
from RootInteractive.InteractiveDrawing.bokeh.bokehDrawHisto import *

from bokeh.io import output_notebook
from RootInteractive.Tools.Alice.BetheBloch import *
output_notebook()

# Setup Toy MC
# Make Toy MC generatinc array of dEdx measurement using Tools MC

In [ ]:
detectors=["ITS","TPC0","TPC1","TPC2","TRD"]
df=toyMC(100000,detectors)
df.head(10)

# Creating the histogram Array

In [ ]:
histogramArray={}
for det in detectors:
    hisString=det+":p:particle:#"+det+">0>>his"+det+"PP(100,0.5,3,100,0.3,5,5,0,5)"
    ddHis=makeHistogram(df,hisString)
    histogramArray[det]=ddHis
 

# Define the parser

# Expression Evaluater

In [ ]:
def evalHistoExpression2(expression, histogramArray):
    # expression  hisdY-hisdZ, abs(hisdY-hisdZ)
    print(expression)
    histogram={}
    axes=[]
    varNames=[]
    query=expression[0][0]
    keys=list(set(re.findall(r"\w+",expression[0][0])).intersection(list(histogramArray.keys())))
    func_list = set(re.findall(r"\w+\(",expression[0][0]))  # there still a paranthesis at the end
    
    for iKey in keys:
        query=query.replace(iKey,"histogramArray[\'"+iKey+"\'][\'H\']")
        
        for i,var in enumerate(histogramArray[iKey]["varNames"]):
            try:
                varNames[i].append(var)
            except:
                varNames.append([var])
            varNames[i]=list(set(varNames[i]))
                           
        axes.append(histogramArray[iKey]["axes"])
    
    
    tmp=axes[0]
    for axe in axes:
        for i in range(len(tmp)):
            if not (axe[i] == tmp[i]).all():
                raise valueError("histograms have incompatible axeses." )
    axes=tmp
    
    for iFunc in func_list:
        if iFunc[:-1] in dir(np):
            query=query.replace(iFunc,"np."+iFunc)  
    
    for i,var in enumerate(varNames):
        varNames[i]=','.join(var)
        
    
    print(query)        
    histogram["H"] = eval(query+"[np.index_exp"+str(expression[0][1]).replace("'","")+"]")
    histogram["name"]=expression[0][0][1:-1]
    print(varNames)
    histogram["varNames"]=varNames
    histogram["axes"]=axes
    
    return histogram

# Draw the histogram

In [ ]:
expressionList=parseProjectionExpression("( (TRD-TRD*0.5+ITS-TRD/2) (0:100,1:10,0:10:2) (0,1) () )")
#expression=parseExpression("( (ITS) (0:100,0:10,0:10) (0,1) () )")
print(expressionList)
histo=evalHistoExpression(expressionList, histogramArray)

In [ ]:
p,d=bokehDrawHistoSliceColz(histo,eval("np.index_exp["+str(expressionList[0][1][0])+"]"), 0,1,1, {'plot_width':800, 'plot_height':700},{'size':10})
show(p)

In [ ]:
p,d = drawHistogramExpression("( (TRD-TRD*0.5+ITS-TRD/2-ITS*0.9999+-ITS*0.0001)(0:100,1:10,0:10:2) (0,1)())",histogramArray,{'plot_width':800, 'plot_height':700},{'size':10})
show(p)